# RAG Assignment: CSV-Based Question Answering System

## Objective
To develop a **Retrieval-Augmented Generation (RAG)** system that can answer questions using a given **CSV dataset**. The system retrieves the most relevant records from the dataset and then generates a final answer based only on the retrieved context.

## Workflow (RAG Pipeline)
**CSV Dataset → Row-to-Text Conversion → Chunking → Embeddings (MiniLM) → Vector Search (FAISS) → Answer Generation (FLAN-T5)**

## Tools / Libraries Used
- **Google Colab** (execution environment)  
- **pandas, numpy** (data handling & preprocessing)  
- **sentence-transformers** (text embeddings)  
- **FAISS** (vector database for retrieval)  
- **transformers** (LLM for response generation)  
- **Gradio (Optional)** (UI for bonus)

> Run the notebook cells in order and upload the CSV file when prompted.



In [1]:
!pip -q install pandas numpy faiss-cpu sentence-transformers accelerate beautifulsoup4 sentencepiece "transformers<5" gradio

import transformers
print("Transformers version:", transformers.__version__)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 11.8 MB/s eta 0:00:00
Transformers version: 4.57.6


## Upload CSV


In [2]:
from google.colab import files
uploaded = files.upload()   # upload your CSV file
csv_path = list(uploaded.keys())[0]
print("Using file:", csv_path)


Saving bright_automotive_company.csv to bright_automotive_company.csv
Using file: bright_automotive_company.csv


## Load and clean dataset


In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv(csv_path)

# Basic cleaning
df = df.replace("?", np.nan)
if "Gender" in df.columns:
    df["Gender"] = df["Gender"].replace({"Femal": "Female"})  # typo fix if present

# Convert numeric columns safely (if these exist)
num_cols = ["Age","Salary","Partner_salary","Total_salary","Price","No_of_Dependents"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Fill missing values
df = df.fillna("Unknown")

print("Shape:", df.shape)
df.head(5)


Shape: (1581, 14)


,Age,Gender,Profession,Marital_status,Education,No_of_Dependents,Personal_loan,House_loan,Partner_working,Salary,Partner_salary,Total_salary,Price,Make
0,53,Male,Business,Married,Post Graduate,4.0,No,No,Yes,99300.0,70700.0,170000,61000,SUV
1,53,Female,Salaried,Married,Post Graduate,4.0,Yes,No,Yes,95500.0,70300.0,165800,61000,SUV
2,53,Female,Salaried,Married,Post Graduate,3.0,No,No,Yes,97300.0,60700.0,158000,57000,SUV
3,53,Female,Salaried,Married,Graduate,Unknown,Yes,No,Yes,72500.0,70300.0,142800,61000,Unknown
4,53,Male,Unknown,Married,Post Graduate,3.0,No,No,Yes,79700.0,60200.0,139900,57000,SUV


## Convert each row into a text chunk (1 row = 1 chunk)


In [4]:
def row_to_text(r):
    return (
        f"Customer: Age={r['Age']}, Gender={r['Gender']}, Profession={r['Profession']}, "
        f"Marital_status={r['Marital_status']}, Education={r['Education']}, Dependents={r['No_of_Dependents']}. "
        f"Loans: Personal_loan={r['Personal_loan']}, House_loan={r['House_loan']}, Partner_working={r['Partner_working']}. "
        f"Income: Salary={r['Salary']}, Partner_salary={r['Partner_salary']}, Total_salary={r['Total_salary']}. "
        f"Car: Make={r['Make']}, Price={r['Price']}."
    )

row_docs = [row_to_text(row) for _, row in df.iterrows()]
row_meta = [{"type":"row", "row_id": int(i), "make": df.loc[i, "Make"]} for i in range(len(df))]

print("Example row chunk:\n", row_docs[0][:600])
print("Total row chunks:", len(row_docs))


Example row chunk:
 Customer: Age=53, Gender=Male, Profession=Business, Marital_status=Married, Education=Post Graduate, Dependents=4.0. Loans: Personal_loan=No, House_loan=No, Partner_working=Yes. Income: Salary=99300.0, Partner_salary=70700.0, Total_salary=170000. Car: Make=SUV, Price=61000.
Total row chunks: 1581


## Create summary chunks (Make-wise stats) for better analytical answers


In [5]:
df_stats = df.copy()

# Convert back numeric for stats (Unknown -> NaN)
for c in ["Salary","Partner_salary","Total_salary","Price","Age","No_of_Dependents"]:
    if c in df_stats.columns:
        df_stats[c] = pd.to_numeric(df_stats[c].replace("Unknown", np.nan), errors="coerce")

if "Make" in df_stats.columns:
    df_stats["Make"] = df_stats["Make"].replace("?", np.nan)

def pct_yes(series):
    s = series.astype(str).str.lower().str.strip()
    return round((s == "yes").mean() * 100, 2)

summary_docs, summary_meta = [], []

if "Make" in df_stats.columns:
    for mk, g in df_stats.groupby("Make"):
        if pd.isna(mk):
            continue
        doc = (
            f"SUMMARY Make={mk}: count={len(g)}; "
            f"avg_price={round(g['Price'].mean(),2)}; median_price={round(g['Price'].median(),2)}; "
            f"min_price={round(g['Price'].min(),2)}; max_price={round(g['Price'].max(),2)}; "
            f"avg_total_salary={round(g['Total_salary'].mean(),2)}; avg_age={round(g['Age'].mean(),2)}; "
            f"personal_loan_yes%={pct_yes(g['Personal_loan'])}; house_loan_yes%={pct_yes(g['House_loan'])}."
        )
        summary_docs.append(doc)
        summary_meta.append({"type":"summary", "row_id": None, "make": mk})

docs_all = row_docs + summary_docs
meta_all = row_meta + summary_meta

print("Total docs (rows + summaries):", len(docs_all))
print("Summary docs:", len(summary_docs))
print("Example summary:\n", summary_docs[0] if summary_docs else "No summaries created.")


Total docs (rows + summaries): 1585
Summary docs: 4
Example summary:
 SUMMARY Make=Hatchback: count=582; avg_price=26474.23; median_price=27000.0; min_price=18000; max_price=33000; avg_total_salary=71018.04; avg_age=25.73; personal_loan_yes%=50.0; house_loan_yes%=40.38.


/tmp/ipython-input-651213260.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_stats[c] = pd.to_numeric(df_stats[c].replace("Unknown", np.nan), errors="coerce")


## Create embeddings


In [6]:
from sentence_transformers import SentenceTransformer

embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
emb = embed_model.encode(docs_all, convert_to_numpy=True, show_progress_bar=True).astype("float32")

print("Embeddings shape:", emb.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/50 [00:00<?, ?it/s]

Embeddings shape: (1585, 384)


## Build FAISS vector index (cosine similarity)


In [7]:
import faiss

faiss.normalize_L2(emb)
index = faiss.IndexFlatIP(emb.shape[1])
index.add(emb)

print("Vectors indexed:", index.ntotal)


Vectors indexed: 1585


## Retrieval function (Top-k)


In [8]:
def retrieve(query, top_k=5):
    q = embed_model.encode([query], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q)
    scores, ids = index.search(q, top_k)

    results = []
    for rank, idx in enumerate(ids[0]):
        results.append({
            "score": float(scores[0][rank]),
            "text": docs_all[idx],
            "meta": meta_all[idx]
        })
    return results

# Quick test
r = retrieve("average price of SUV", top_k=3)
for i, item in enumerate(r, 1):
    print(f"\n--- Retrieved {i} | score={item['score']:.3f} | meta={item['meta']} ---")
    print(item["text"][:350], "...")



--- Retrieved 1 | score=0.546 | meta={'type': 'summary', 'row_id': None, 'make': 'SUV'} ---
SUMMARY Make=SUV: count=295; avg_price=57681.55; median_price=57000.0; min_price=58; max_price=680000; avg_total_salary=96478.64; avg_age=44.17; personal_loan_yes%=38.98; house_loan_yes%=7.46. ...

--- Retrieved 2 | score=0.393 | meta={'type': 'summary', 'row_id': None, 'make': 'Sedan'} ---
SUMMARY Make=Sedan: count=702; avg_price=34603.99; median_price=33000.0; min_price=18000; max_price=55000; avg_total_salary=79584.76; avg_age=31.92; personal_loan_yes%=54.84; house_loan_yes%=38.46. ...

--- Retrieved 3 | score=0.340 | meta={'type': 'summary', 'row_id': None, 'make': 'Hatchback'} ---
SUMMARY Make=Hatchback: count=582; avg_price=26474.23; median_price=27000.0; min_price=18000; max_price=33000; avg_total_salary=71018.04; avg_age=25.73; personal_loan_yes%=50.0; house_loan_yes%=40.38. ...


## Generator (FLAN-T5) and RAG answer function


In [9]:
from transformers import pipeline

generator = pipeline("text2text-generation", model="google/flan-t5-base")

def rag_answer(query, top_k=5):
    retrieved = retrieve(query, top_k=top_k)

    # Keep prompt small to avoid truncation
    context = "\n\n".join([f"- {r['text'][:800]}" for r in retrieved])

    prompt = (
        "Answer using ONLY the context below. "
        "If the answer is not in the context, say: Not found in the provided dataset.\n\n"
        f"CONTEXT:\n{context}\n\n"
        f"QUESTION: {query}\n\nANSWER:"
    )

    out = generator(prompt, max_new_tokens=220)[0]["generated_text"]
    return out, retrieved


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


## Minimum 3 test queries (Required)


In [10]:
test_queries = [
    "Which car make has the highest average price?",
    "What is the loan pattern for SUV buyers?",
    "Give a typical customer profile for Sedan buyers."
]

for q in test_queries:
    ans, src = rag_answer(q, top_k=5)
    print("\n==============================")
    print("Query:", q)
    print("Answer:", ans)
    print("Top sources (type, make, score):", [(s["meta"].get("type"), s["meta"].get("make"), round(s["score"],3)) for s in src])


Token indices sequence length is longer than the specified maximum sequence length for this model (580 > 512). Running this sequence through the model will result in indexing errors



Query: Which car make has the highest average price?
Answer: Make=Sedan
Top sources (type, make, score): [('summary', 'Sedan', 0.614), ('summary', 'SUV', 0.582), ('summary', 'Hatchback', 0.495), ('summary', 'Unknown', 0.475), ('row', 'Sedan', 0.388)]

Query: What is the loan pattern for SUV buyers?
Answer: Personal_loan=Yes, House_loan=No, Partner_working=Yes. Income: Salary=99300.0, Partner_salary=70000.0, Total_salary=169300. Car: Make=SUV, Price=66000.
Top sources (type, make, score): [('summary', 'SUV', 0.616), ('row', 'SUV', 0.563), ('row', 'SUV', 0.563), ('row', 'SUV', 0.562), ('row', 'SUV', 0.558)]

Query: Give a typical customer profile for Sedan buyers.
Answer: Gender=Male, Profession=Salaried, Marital_status=Married, Education=Graduate, Dependents=1.0. Loans: Personal_loan=No, House_loan=No, Partner_working=No. Income: Salary=57400.0, Partner_salary=0.0, Total_salary=101600. Car: Make=Sedan, Price=47000
Top sources (type, make, score): [('row', 'Sedan', 0.496), ('row', 'Seda

## Summary
This assignment successfully implements a **Retrieval-Augmented Generation (RAG)** system on a **CSV dataset**. The dataset records are converted into text chunks and stored as embeddings using **MiniLM**. A **FAISS** vector index is used to retrieve the most relevant chunks for each user query, and **FLAN-T5** generates the final answer using only the retrieved context.  
The system was tested using multiple queries and produced grounded, dataset-based responses. Future improvements can include better chunking for tabular data, reranking/hybrid search for higher retrieval accuracy, metadata-based filtering (Make/Profession/Income), and a complete UI using Gradio or Streamlit for end-user interaction.
